In [24]:
import pandas as pd
pd.options.display.max_colwidth = 160

import numpy as np

from openai import OpenAI
from transformers import GPT2Tokenizer

import a_env_vars
import os
os.environ["OPENAI_API_KEY"] = a_env_vars.api_key

In [2]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [3]:
# get complete data and create small tables to uses as source for the open ai agent

In [4]:
df = pd.read_csv("Walmart_Sales.csv")
df = df.query("Store<6") # only 5 stores
df["Date"] = pd.to_datetime(df['Date'], format="%d-%m-%Y")
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 715 entries, 0 to 714
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         715 non-null    int64         
 1   Date          715 non-null    datetime64[ns]
 2   Weekly_Sales  715 non-null    float64       
 3   Holiday_Flag  715 non-null    int64         
 4   Temperature   715 non-null    float64       
 5   Fuel_Price    715 non-null    float64       
 6   CPI           715 non-null    float64       
 7   Unemployment  715 non-null    float64       
dtypes: datetime64[ns](1), float64(5), int64(2)
memory usage: 50.3 KB


In [5]:
# Create a table with sales totals and variations by store, comparison of current vs last week

class CreateTable:
    def __init__(self, df, current_week, last_week):
        self.df = df.sort_values(by="Date")
        self.current_week = current_week
        self.last_week = last_week

    def create_summary_table(self):
        self.result_df = self.df[self.df["Date"].isin([self.current_week, self.last_week])]
        
        # Pivot table
        self.result_df = self.result_df.pivot_table(index='Store', columns="Date", values=["Weekly_Sales"])
        self.result_df.columns = ["Previous Week", "Current Week"]
        self.result_df.reset_index(inplace=True)

        # Total row
        total_row = pd.DataFrame([{
                        "Store" : "Total",
                        "Previous Week":  self.result_df["Previous Week"].sum(),
                        "Current Week":  self.result_df["Current Week"].sum()
                         }]
                    )

        self.result_df = pd.concat([self.result_df, total_row])

        # Variation columns
        self.result_df["Weekly Variation"] = np.round(self.result_df["Current Week"] - self.result_df["Previous Week"], 0)
        self.result_df["Weekly Variation %"] = np.round((self.result_df["Weekly Variation"] / self.result_df["Previous Week"]) * 100, 2)
        self.result_df["Change Type"] = np.where(self.result_df["Weekly Variation"]>0, "increase", "decrease")


        self.result_df = self.result_df.sort_values(by="Current Week",ascending=False).reset_index(drop=True)

        return self.result_df
    
    def create_summary_text(self):
        summary_list = []

        for idx in self.result_df.index:
            if idx == 0:
                text = "Total Sales changed in {} dolars wich represents a {} of {} percent. \n".format(self.result_df.at[idx, "Weekly Variation"], self.result_df.at[idx, "Change Type"], self.result_df.at[idx, "Weekly Variation %"])
                #print(text)
            else:
                text = "Sales in store {} changed {} dolars wich represents a {} of {} percent.\n".format(self.result_df.at[idx, "Store"], self.result_df.at[idx, "Weekly Variation"], self.result_df.at[idx, "Change Type"], self.result_df.at[idx, "Weekly Variation %"])
                #print(text)

            summary_list.append(text)
            summary_text = ''.join(summary_list)

        return summary_text


In [10]:
class CreateHighlights:
    def __init__(self, text, model="gpt-3.5-turbo") -> None:
        self.text = text
        self.model = model
        self.prompt = f"""
            You will be provided with text delimited by triple quotes about sales in a retail store chain. 

            when total sales increased, write:

            - Total sales increased by (value $) (value %). The increase was driven mainly by an expansion of (value $) (value %) in the store (store with highest increase).

            when total sales decreased, change the sense of the sentences:

            \"\"\"{self.text}\"\"\"
        """

    def count_tokens(self, text_to_ctokens):
        self.text_to_ctokens = text_to_ctokens
        tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
        tokens = tokenizer.encode(self.text_to_ctokens)
        return len(tokens)


    def get_highlights(self):
        # Message to return
        messages = [{"role": "user", "content": self.prompt}]
        response = client.chat.completions.create(
            model=self.model,
            messages=messages,
            temperature=0
        )

        self.returned_message = response.choices[0].message.content
        return self.returned_message
    
    def used_tokes(self):
        self.prompt_tokens = self.count_tokens(self.prompt)
        self.returned_tokens =  self.count_tokens(self.returned_message)
        self.total_tokens =  self.prompt_tokens + self.returned_tokens
        #print("Prompt tokens:", self.count_tokens(self.prompt), "Message tokens:", self.count_tokens(self.returned_message), "Total:", self.total_tokens)
        return self.total_tokens
    
    def total_cost(self):
        """
        Given the cost:
        - Input: 0,50 US$ / 1M tokens
        - Output: 1,50 US$ / 1M tokens
        """
        self.input_cost = self.prompt_tokens * 0.5 / 1000000 # prompt cost
        self.output_cost = self.returned_tokens * 1.5 / 1000000 # prompt cost
        self.total_cost = self.input_cost + self.output_cost
        return self.total_cost
    


In [7]:
summary = CreateTable(df, "2012-10-12", "2012-10-05")
summ_table = summary.create_summary_table()
summ_text = summary.create_summary_text()# Get the summary

print(summ_table.drop("Change Type", axis=1))
print(summ_text)

   Store  Previous Week  Current Week  Weekly Variation  Weekly Variation %
0  Total     6665548.38    6342993.81         -322555.0               -4.84
1      4     2209835.43    2133026.07          -76809.0               -3.48
2      2     1998321.04    1900745.13          -97576.0               -4.88
3      1     1670785.97    1573072.81          -97713.0               -5.85
4      3      443557.65     410804.39          -32753.0               -7.38
5      5      343048.29     325345.41          -17703.0               -5.16
Total Sales changed in -322555.0 dolars wich represents a decrease of -4.84 percent. 
Sales in store 4 changed -76809.0 dolars wich represents a decrease of -3.48 percent.
Sales in store 2 changed -97576.0 dolars wich represents a decrease of -4.88 percent.
Sales in store 1 changed -97713.0 dolars wich represents a decrease of -5.85 percent.
Sales in store 3 changed -32753.0 dolars wich represents a decrease of -7.38 percent.
Sales in store 5 changed -17703.0 dola

In [11]:
# Get highlights
highl_t = CreateHighlights(summ_text)
print(highl_t.get_highlights())
print("Total tokens:", highl_t.used_tokes())
print("Total Cost:", highl_t.total_cost())

Total sales decreased by $322,555.0 (4.84%). The decrease was driven mainly by a reduction of $97,713.0 (5.85%) in store 1.
Total tokens: 340
Total Cost: 0.00020899999999999998


### Runnig the highlights for all the weeks and estimate the cost

In [12]:
# Small simulation and analysis of tokens used by week

df.sort_values(by="Date", inplace=True) # Sort by date ascending
df["Date"] = df["Date"].apply(lambda x: format(x, "")[:10]) # covert back to string

In [29]:
# Get the pairs of weeks (current, previous)
weeks_lists = [i for i in df["Date"].unique()]
week_tuples = [(weeks_lists[i], weeks_lists[i-1]) for i in range(1,len(weeks_lists))]
print(len(week_tuples))



# Run a save results
highlights_list = [] 

for week_pair in week_tuples:
    current_week = week_pair[0]
    previous_week = week_pair[1]

    summary = CreateTable(df, current_week, previous_week)
    summ_table = summary.create_summary_table()
    summ_text = summary.create_summary_text()

    highl_t = CreateHighlights(summ_text)
    high_text = highl_t.get_highlights()
    highl_t.used_tokes()
    high_cost = highl_t.total_cost()

    highlights_list.append({
        "Week" : current_week,
        "Highlights" : high_text,
        "Total Cost": high_cost
    })

    

142


In [31]:
highlights_df = pd.DataFrame(highlights_list)
print("Total Cost", highlights_df["Total Cost"].sum())
highlights_df.sample(20)

Total Cost 0.032588000000000006


,Week,Highlights,Total Cost
115,2012-04-27,Total sales decreased by 261561.0 dollars (4.17%). The decrease was driven mainly by a reduction of 83211.0 dollars (4.5%) in store 2.,0.000209
128,2012-07-27,Total sales decreased by 304898.0 dollars (4.88%). The decrease was driven mainly by a reduction of 126950.0 dollars (5.84%) in store 4.,0.000209
8,2010-04-09,Total sales decreased by 332652.0 dollars (5.2%). The decrease was driven mainly by a reduction of 160794.0 dollars (8.12%) in store 4.,0.000209
81,2011-09-02,Total sales increased by 178374.0 dollars (3.0%). The increase was driven mainly by an expansion of 89930.0 dollars (4.52%) in the store 4.\n\nSales in stor...,0.000309
125,2012-07-06,Total sales increased by 485828.0 dolars (7.71%). The increase was driven mainly by an expansion of 229433.0 dolars (14.89%) in the store 1.,0.000209
33,2010-10-01,Total sales increased by 229762.0 dollars (4.15%). The increase was driven mainly by an expansion of 101538.0 dollars (7.51%) in the store 1.,0.000202
28,2010-08-27,"Total sales decreased by $301,489 (-4.93%). The decrease was driven mainly by a reduction of $148,777 (-7.45%) in store 4.",0.000200
129,2012-08-03,Total sales increased by 554024.0 dollars (9.32%). The increase was driven mainly by an expansion of 192012.0 dollars (13.34%) in the store 1.,0.000203
90,2011-11-04,Total sales increased by 688470.0 dollars (11.36%). The increase was driven mainly by an expansion of 251980.0 dollars (17.44%) in the store 1.,0.000204
69,2011-06-10,Total sales decreased by 69228.0 dollars (-1.09%). The decrease was driven mainly by a reduction of 46130.0 dollars (-2.82%) in the store 1.,0.000207
